In [21]:
import geopandas as gpd
import pandas as pd
import pydeck as pdk
import warnings
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

mpl.use('TkAgg')  # !IMPORTANT

def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [22]:
indo = gpd.read_file('FIX_INA_Pupuk.shp')
# ax = indo.geometry.plot(figsize=(10,10), color='none', edgecolor='gainsboro', zorder=3)
# embed_map

In [23]:
myl = list(indo.MEAN)
myl.sort()
print(myl)
med = len(myl) // 2
print("Median of Mean Data: {}".format(myl[med]))

[1.20508413669, 1.38054625314, 1.44404702931, 1.50355371583, 1.50389812964, 1.50595387766, 1.51236026076, 1.51467413353, 1.51576116908, 1.51706247814, 1.51707161853, 1.53202599956, 1.53222020453, 1.53515771432, 1.56428020236, 1.56512209308, 1.5754733483, 1.57652300024, 1.59237447767, 1.59986514131, 1.60138642408, 1.62038136136, 1.63154578265, 1.6432110722, 1.66180367047, 1.68508420633, 1.83200111759, 1.89718463245, 1.92323040858, 1.98236163249, 1.99016524099, 2.00364606463, 2.04198140907, 2.0522960072]
Median of Mean Data: 1.57652300024


In [24]:
# pupuk, demand = list(indo.Dosis_Pupu), list(indo.Total_kebu)
pupuk = list(indo.Data_untuk)
print(pupuk)
pupuk_conv = [float(x.replace('.', '').replace(',','.')) for x in pupuk]
# indo.Dosis_Pupu = pupuk_conv
indo.Data_untuk = pupuk_conv
pupuk_conv.sort()
# demand.sort()

# print(pupuk_conv)
# print(demand)
print("Median of Pupuk Hortikultura Data: {}".format(pupuk_conv[med]))
# print("Median of Pupuk Perkebunan Data: {}".format(demand[med]))

# indo.Data_untuk

['10.912,605', '11.890,725', '28.961,973', '4.128,300', '8.604,495', '170.061,195', '27.494,779', '43.645,905', '66.590,070', '35.870,978', '30.417,492', '565.858,275', '97.329,540', '212.954,212', '84.283,924', '25.408,635', '665.050,815', '34.391,640', '30.274,530', '87.432,090', '10.995,495', '107.379,765', '22.015,080', '19.852,080', '35.460,690', '17.202,060', '427.398,068', '28.887,120', '183.798,482', '17.989,095', '28.344,015', '19.942,470', '31.693,620', '34.986,900']
Median of Pupuk Hortikultura Data: 31693.62


In [25]:
indo.loc[indo.Provinsi == 'Jakarta Raya', 'Provinsi'] = 'DKI Jakarta'
indo.loc[indo.Provinsi == 'Riau', 'Provinsi'] = 'Provinsi Riau'

print((indo.Provinsi))

0         Bangka Belitung
1               Gorontalo
2           Provinsi Riau
3             DKI Jakarta
4          Kepulauan Riau
5        Sulawesi Selatan
6        Sumatera Selatan
7        Kalimantan Barat
8                    Aceh
9                Bengkulu
10                Lampung
11            Jawa Tengah
12    Nusa Tenggara Barat
13         Sumatera Utara
14                  Jambi
15                 Banten
16             Jawa Timur
17             Yogyakarta
18     Kalimantan Selatan
19         Sulawesi Utara
20       Kalimantan Utara
21         Sulawesi Barat
22           Maluku Utara
23      Kalimantan Tengah
24    Nusa Tenggara Timur
25            Papua Barat
26             Jawa Barat
27      Sulawesi Tenggara
28         Sumatera Barat
29                  Papua
30       Kalimantan Timur
31                 Maluku
32                   Bali
33        Sulawesi Tengah
Name: Provinsi, dtype: object


In [26]:
# Union dari semuanya harus berjumlah 33
supplier = set(['Jawa Barat', 'Jawa Timur', 'Kalimantan Timur','Sumatera Selatan', 'Aceh'])

a = set(indo.loc[(indo.MEAN >= myl[med]) & (indo.Data_untuk >= pupuk_conv[med])].Provinsi)
high_prior = a.difference(supplier)
# index bagus, demand high (prioritas ke 1)

b1 = set(indo.loc[(indo.MEAN > myl[med]) & (indo.Data_untuk < pupuk_conv[med])].Provinsi)
b2 = set(indo.loc[(indo.MEAN < myl[med]) & (indo.Data_untuk > pupuk_conv[med])].Provinsi)
med_prior = (b1.union(b2)).difference(supplier)
# prioritas ke 2

d = set(indo.loc[(indo.MEAN < myl[med]) & (indo.Data_untuk < pupuk_conv[med])].Provinsi)
low_prior = d.difference(supplier)
# prioritas ke 3

# full_olah = ((supplier.union(high_prior)).union(med_prior)).union(low_prior)
# print(full.difference(full_olah))
print(supplier)
print(high_prior)
print(med_prior)
print(low_prior)
print(len(supplier) + len(high_prior) + len(med_prior) + len(low_prior))

high = list(high_prior)
mid = list(med_prior)
low = list(low_prior)
supplier = list(supplier)

{'Aceh', 'Sumatera Selatan', 'Jawa Timur', 'Kalimantan Timur', 'Jawa Barat'}
{'Nusa Tenggara Timur', 'Nusa Tenggara Barat', 'Yogyakarta', 'Sulawesi Tengah', 'Jambi', 'Bali', 'Sulawesi Selatan', 'Jawa Tengah'}
{'Sulawesi Barat', 'Kalimantan Barat', 'Provinsi Riau', 'Bengkulu', 'Sumatera Barat', 'Lampung', 'Banten', 'Maluku Utara', 'Sulawesi Utara', 'Sulawesi Tenggara', 'Gorontalo', 'Maluku', 'Sumatera Utara'}
{'DKI Jakarta', 'Kalimantan Selatan', 'Kalimantan Utara', 'Papua', 'Papua Barat', 'Kepulauan Riau', 'Kalimantan Tengah', 'Bangka Belitung'}
34


In [27]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [28]:
def stroke_factory(val):
    # print(val)
    if val <= 100:
        return 75
    elif val <= 500:
        return 60
    elif val <= 1000:
        return 50
    elif val <= 2500:
        return 40
    elif val <= 5000:
        return 30
    elif val <= 10000:
        return 20
    else:
        return 10

In [29]:
# Get land coordinates
indo_provinces = pd.read_csv('daftar_nama_daerah.csv')
indo_provinces = indo_provinces.iloc[:33]
# indo_provinces.head()

kaltara_lng = float(indo.loc[indo.Provinsi == 'Kalimantan Utara'].geometry.centroid.x) 
kaltara_lat = float(indo.loc[indo.Provinsi == 'Kalimantan Utara'].geometry.centroid.y)
# print(kaltara_lng,kaltara_lat, len(high), len(mid), len(low))

maluku_lng = float(indo_provinces.loc[indo_provinces.name == 'Provinsi Maluku'].longitude)
maluku_lat = float(indo_provinces.loc[indo_provinces.name == 'Provinsi Maluku'].latitude)
maltara_lng = float(indo_provinces.loc[indo_provinces.name == 'Provinsi Maluku Utara'].longitude)
maltara_lat = float(indo_provinces.loc[indo_provinces.name == 'Provinsi Maluku Utara'].latitude)

# print(maluku_lat, maluku_lng, maltara_lat, maltara_lng)
papua_lng = float(indo_provinces.loc[indo_provinces.name == 'Provinsi Papua'].longitude)
papua_lat = float(indo_provinces.loc[indo_provinces.name == 'Provinsi Papua'].latitude)

# print(papua_lat, papua_lng)

In [30]:
# Get high priority land coordinate
high_coords_lng, high_coords_lat = [], []
for province in high:
    for idx in range(len(indo_provinces)):
        # print(province, indo_provinces.iloc[idx]['name'])
        if province in indo_provinces.iloc[idx]['name']:
            high_coords_lng.append(indo_provinces.iloc[idx].longitude)
            high_coords_lat.append(indo_provinces.iloc[idx].latitude)

print(high_coords_lng, high_coords_lat)
print(len(high_coords_lng))

# Get middle priority land coordinate
mid_coords_lng, mid_coords_lat = [], []
for province in mid:
    if province != 'Maluku' and province != 'Maluku Utara':
        for idx in range(len(indo_provinces)):
            if province in indo_provinces.iloc[idx]['name']:
                mid_coords_lng.append(indo_provinces.iloc[idx].longitude)
                mid_coords_lat.append(indo_provinces.iloc[idx].latitude)
    else:
        if province == 'Maluku':
            mid_coords_lng.append(maluku_lng)
            mid_coords_lat.append(maluku_lat)
        elif province == 'Maluku Utara':
            mid_coords_lng.append(maltara_lng)
            mid_coords_lat.append(maltara_lat)

print(mid_coords_lng, mid_coords_lat)
print(len(mid_coords_lng))

# Get middle priority land coordinate
low_coords_lng, low_coords_lat = [], []
for province in low:
    if province != 'Kalimantan Utara' and province != 'Papua':
        for idx in range(len(indo_provinces)):
            if province in indo_provinces.iloc[idx]['name']:
                low_coords_lng.append(indo_provinces.iloc[idx].longitude)
                low_coords_lat.append(indo_provinces.iloc[idx].latitude)
    else:
        if province == 'Kalimantan Utara':
            low_coords_lng.append(kaltara_lng)
            low_coords_lat.append(kaltara_lat)
        elif province == 'Papua':
            low_coords_lng.append(papua_lng)
            low_coords_lat.append(papua_lat)

print(low_coords_lng, low_coords_lat)
print(len(low_coords_lat))

# Get distributor region land coordinate
supp_lng, supp_lat = [], []
for province in supplier:
    for idx in range(len(indo_provinces)):
        # print(province, indo_provinces.iloc[idx]['name'])
        if province in indo_provinces.iloc[idx]['name']:
            supp_lng.append(indo_provinces.iloc[idx].longitude)
            supp_lat.append(indo_provinces.iloc[idx].latitude)

print(supp_lng, supp_lat)
print(len(supp_lng))


[121.0793705, 117.3616476, 110.4262088, 121.4456179, 102.4380581, 115.188916, 119.9740534, 110.1402594] [-8.6573819, -8.6529334, -7.8753849, -1.4300254, -1.4851831, -8.4095178, -3.6687994, -7.150975]
8
[119.2320784, 111.4752851, 101.7068294, 102.3463875, 100.8000051, 105.4068079, 106.0640179, 127.8087693, 123.9750018, 122.174605, 122.4467238, 130.1452734, 99.5450974] [-2.8441371, -0.2787808, 0.2933469, -3.5778471, -0.7399397, -4.5585849, -6.4058172, 1.5709993, 0.6246932, -4.14491, 0.6999372, -3.2384616, 2.1153547]
13
[106.845172, 115.2837585, 116.21634789778791, 138.0803529, 133.1747162, 108.1428669, 113.3823545, 106.4405872] [-6.211544, -3.0926415, 2.8888907926569765, -4.269928, -1.3361154, 3.9456514, -1.6814878, -2.7410513]
8
[96.7493993, 103.914399, 112.2384017, 116.419389, 107.668887] [4.695135, -3.3194374, -7.5360639, 1.6406296, -7.090911]
5


In [17]:
supp_lng

[96.7493993, 103.914399, 112.2384017, 116.419389, 107.668887]

In [31]:
list_of_gdf1, list_of_gdf2, list_of_gdf3= [], [], []
df = None
points = 100

# Creating curved coordinates using sine and cosine functions - Not implemented 
curve_factor = 0.5 
t = np.linspace(0, 1, points)

for i in range(len(supplier)):
    from_x = supp_lng[i]
    from_y = supp_lat[i]
                    
    to_coords_1, distances_1, stroke_1, c_lat_1, c_lon_1 = [], [], [], [], []
    for j in range(len(high)):
        lng = high_coords_lng[j]
        lat = high_coords_lat[j]
        to_coords_1.append([lng,lat])
        distances_1.append(int(haversine(from_y, from_x, lat, lng)))
        stroke_1.append(stroke_factory(distances_1[j]))
        mid_lat = (from_y + lat) / 2
        mid_lon = (from_x + lng) / 2
        c_lat_1.append(curve_factor * np.sin(t * np.pi) + mid_lat)
        c_lon_1.append(curve_factor * np.cos(t * np.pi) + mid_lon)

    list_of_gdf1.append(pd.DataFrame({
        'from' : supplier[i],
        'from_x' : from_x,
        'from_y' : from_y,
        'to' : high,
        'to_coords' : to_coords_1,
        'distance(km)' : distances_1,
        'stroke_val' : stroke_1,
        # 'curve_lat' : c_lat_1,
        # 'curve_lon' : c_lon_1
    }))

    to_coords_2, distances_2, stroke_2, c_lat_2, c_lon_2= [], [], [], [], []
    for j in range(len(mid)):
        lng = mid_coords_lng[j]
        lat = mid_coords_lat[j]
        to_coords_2.append([lng,lat])
        distances_2.append(int(haversine(from_y, from_x, lat, lng)))
        stroke_2.append(stroke_factory(distances_2[j]))
        mid_lat = (from_y + lat) / 2
        mid_lon = (from_x + lng) / 2
        c_lat_2.append(curve_factor * np.sin(t * np.pi) + mid_lat)
        c_lon_2.append(curve_factor * np.cos(t * np.pi) + mid_lon)


    list_of_gdf2.append(pd.DataFrame({
        'from' : supplier[i],
        'from_x' : from_x,
        'from_y' : from_y,
        'to' : mid,
        'to_coords' : to_coords_2,
        'distance(km)' : distances_2,
        'stroke_val' : stroke_2,
        # 'curve_lat' : c_lat_2,
        # 'curve_lon' : c_lon_2
    }))

    to_coords_3, distances_3, stroke_3, c_lat_3, c_lon_3= [], [], [], [], []
    for j in range(len(low)):
        lng = low_coords_lng[j]
        lat = low_coords_lat[j]
        to_coords_3.append([lng,lat])
        distances_3.append(int(haversine(from_y, from_x, lat, lng)))
        stroke_3.append(stroke_factory(distances_3[j]))
        mid_lat = (from_y + lat) / 2
        mid_lon = (from_x + lng) / 2
        c_lat_3.append(curve_factor * np.sin(t * np.pi) + mid_lat)
        c_lon_3.append(curve_factor * np.cos(t * np.pi) + mid_lon)


    list_of_gdf3.append(pd.DataFrame({
        'from' : supplier[i],
        'from_x' : from_x,
        'from_y' : from_y,
        'to' : low,
        'to_coords' : to_coords_3,
        'distance(km)' : distances_3,
        'stroke_val' : stroke_3,
        # 'curve_lat' : c_lat_2,
        # 'curve_lon' : c_lon_2
    }))

print((list_of_gdf1[0]))
print((list_of_gdf2[0]))
print((list_of_gdf3[0]))
print(len(list_of_gdf1))
print(len(list_of_gdf2))
print(len(list_of_gdf3))
# for i in list_of_gdf2:
#     print(i)

   from     from_x    from_y                   to                  to_coords  \
0  Aceh  96.749399  4.695135  Nusa Tenggara Timur  [121.0793705, -8.6573819]   
1  Aceh  96.749399  4.695135  Nusa Tenggara Barat  [117.3616476, -8.6529334]   
2  Aceh  96.749399  4.695135           Yogyakarta  [110.4262088, -7.8753849]   
3  Aceh  96.749399  4.695135      Sulawesi Tengah  [121.4456179, -1.4300254]   
4  Aceh  96.749399  4.695135                Jambi  [102.4380581, -1.4851831]   
5  Aceh  96.749399  4.695135                 Bali   [115.188916, -8.4095178]   
6  Aceh  96.749399  4.695135     Sulawesi Selatan  [119.9740534, -3.6687994]   
7  Aceh  96.749399  4.695135          Jawa Tengah   [110.1402594, -7.150975]   

   distance(km)  stroke_val  
0          3079          30  
1          2725          30  
2          2062          40  
3          2826          30  
4           933          50  
5          2510          30  
6          2742          30  
7          1985          40  
    from 

In [ ]:
layers_1 = []

for df in list_of_gdf1:
    # print(df)
    layers_1.append(pdk.Layer(
        "GreatCircleLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_arc_source_position="[from_x, from_y]",
        get_arc_target_position="to_coords",
        # get_arc_center="[curve_lon, curve_lat]",
        get_arc_height=100000,
        get_source_color=[255,0,0],
        get_target_color=[255,122,0],
        auto_highlight=True
    ))
    print(layers_1)

layers_2 = []

for df in list_of_gdf2:
    layers_2.append(pdk.Layer(
        "GreatCircleLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_arc_source_position="[from_x, from_y]",
        get_arc_target_position="to_coords",
        # get_arc_center="[curve_lon, curve_lat]",
        get_source_color=[255,122,0],
        get_target_color=[112,255,0],
        get_arc_height=100000,
        auto_highlight=True
    ))

print(layers_1[0].id)

In [331]:
view_state = pdk.ViewState(latitude=0.7893, longitude=113.9213, zoom=3, bearing=0, pitch=50)

r = pdk.Deck(
    layers= layers_2,
    initial_view_state=view_state,
    tooltip={"text": "{from} distribusi ke {to}"},
)
# print(r)
r.picking_radius = 10

r.to_html("prior2.html")

In [ ]:
view_state = pdk.ViewState(latitude=0.7893, longitude=113.9213, zoom=3, bearing=0, pitch=50)

r = pdk.Deck(
    layers= layers_1,
    initial_view_state=view_state,
    tooltip={"text": "{from} distribusi ke {to}"},
)
# print(r)
r.picking_radius = 10

r.to_html("prior2.html")

In [32]:
print(len(list_of_gdf1), len(list_of_gdf2), len(list_of_gdf3))

5 5 5


In [33]:
# Concatenate and Validate the Consistency
gdfs = list_of_gdf1
for el in list_of_gdf2:
    gdfs.append(el)
for el in list_of_gdf3:
    gdfs.append(el)

print(len(list_of_gdf1[0]), len(list_of_gdf2[0]), len(list_of_gdf3[0]))
print(len(gdfs)) 

8 13 8
15


In [34]:
layers_1 = []

for df in list_of_gdf1[0]:
    layers_1.append(pdk.Layer(
        "ArcLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_tilt=15,
        get_source_color=[255,0,0],
        get_target_color=[255,0,0],
        auto_highlight=True
    ))

layers_2 = []

for df in list_of_gdf2:
    layers_2.append(pdk.Layer(
        "ArcLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_tilt=15,
        get_source_color=[255,122,0],
        get_target_color=[255,122,0],
        auto_highlight=True
    ))

layers_3 = []

for df in list_of_gdf3:
    layers_3.append(pdk.Layer(
        "ArcLayer",
        df,
        pickable=True,
        get_stroke_width='stroke_val',
        get_source_position= "[from_x, from_y]",
        get_target_position = "to_coords",
        get_tilt=15,
        get_source_color=[255,214,0],
        get_target_color=[255,214,0],
        auto_highlight=True
    ))

layers = layers_1
for el in layers_2:
    layers.append(el)
for el in layers_3:
    layers.append(el)

view_state = pdk.ViewState(latitude=0.7893, longitude=113.9213, zoom=4, bearing=20, pitch=100)

r = pdk.Deck(
    layers= layers,
    initial_view_state=view_state,
    tooltip={"text": "{from} distribusi ke {to}"},
)
# print(r)
r.picking_radius = 10

r.to_html('prior_arc.html')


In [36]:
layer_1 = (pdk.Layer(
    "ArcLayer",
    list_of_gdf1[0],
    pickable=True,
    get_stroke_width='stroke_val',
    get_source_position= "[from_x, from_y]",
    get_target_position = "to_coords",
    get_tilt=15,
    get_source_color=[255,0,0],
    get_target_color=[255,0,0],
    auto_highlight=True
))

layer_2 = (pdk.Layer(
    "ArcLayer",
    list_of_gdf2[0],
    pickable=True,
    get_stroke_width='stroke_val',
    get_source_position= "[from_x, from_y]",
    get_target_position = "to_coords",
    get_tilt=15,
    get_source_color=[255,122,0],
    get_target_color=[255,122,0],
    auto_highlight=True
))

layer_3 = (pdk.Layer(
    "ArcLayer",
    list_of_gdf3[0],
    pickable=True,
    get_stroke_width='stroke_val',
    get_source_position= "[from_x, from_y]",
    get_target_position = "to_coords",
    get_tilt=15,
    get_source_color=[255,214,0],
    get_target_color=[255,214,0],
    auto_highlight=True
))

view_state = pdk.ViewState(latitude=0.7893, longitude=113.9213, zoom=4, bearing=20, pitch=100)

r = pdk.Deck(
    layers= [layer_1, layer_2, layer_3],
    initial_view_state=view_state,
    tooltip={"text": "{from} supply ke {to}"},
)
# print(r)
r.picking_radius = 10

r.to_html('aceh.html')